In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
import networkx as nx
from pyvis.network import Network

WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"
ORKG_ENDPOINT = "https://orkg.org/sparql"

def find_wikidata_entity(term):
    """Find entity URI in Wikidata matching the label."""
    sparql = SPARQLWrapper(WIKIDATA_ENDPOINT)
    sparql.setQuery(f"""
    SELECT ?item WHERE {{
      ?item rdfs:label "{term}"@en .
    }} LIMIT 1
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    bindings = results["results"]["bindings"]
    return bindings[0]["item"]["value"] if bindings else None

def find_orkg_entities(label):
    """Find ORKG concepts that match or contain the label."""
    sparql = SPARQLWrapper(ORKG_ENDPOINT)
    sparql.setQuery(f"""
    SELECT DISTINCT ?entity ?label WHERE {{
      ?entity rdfs:label ?label .
      FILTER(CONTAINS(LCASE(?label), "{label.lower()}"))
    }} LIMIT 10
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return [(r["entity"]["value"], r["label"]["value"]) for r in results["results"]["bindings"]]

def get_direct_neighbors(entity_uri):
    """Fetch direct RDF neighbors of a given ORKG entity."""
    sparql = SPARQLWrapper(ORKG_ENDPOINT)
    sparql.setQuery(f"""
    SELECT DISTINCT ?neighbor ?label WHERE {{
      {{ <{entity_uri}> ?p ?neighbor . }}
      UNION
      {{ ?neighbor ?p <{entity_uri}> . }}
      ?neighbor rdfs:label ?label .
    }} LIMIT 50
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return [(r["neighbor"]["value"], r["label"]["value"]) for r in results["results"]["bindings"]]

def build_graph_from(start_uri, target_uri, depth=3):
    """Build a graph from ORKG concepts starting at start_uri, searching paths to target_uri."""
    graph = nx.DiGraph()
    visited = set()
    paths_found = []

    def dfs(node, path, current_depth):
        if current_depth > depth or node in visited:
            return
        visited.add(node)
        neighbors = get_direct_neighbors(node)
        for neighbor_uri, label in neighbors:
            graph.add_node(neighbor_uri, label=label)
            graph.add_edge(node, neighbor_uri)
            if neighbor_uri == target_uri:
                paths_found.append(path + [neighbor_uri])
            else:
                dfs(neighbor_uri, path + [neighbor_uri], current_depth + 1)

    dfs(start_uri, [start_uri], 0)
    return graph, paths_found

def visualize_graph(graph, filename="orkg_graph.html"):
    """Render the graph using PyVis and save it to an HTML file."""
    net = Network(height="600px", width="100%", notebook=False)
    for node in graph.nodes:
        label = graph.nodes[node].get("label", node.split("/")[-1])
        net.add_node(node, label=label)
    for edge in graph.edges:
        net.add_edge(edge[0], edge[1])
    net.show(filename)

# ---- MAIN LOGIC ----
term1 = "photosynthesis"
term2 = "artificial intelligence"

print(f"🔍 Recherche Wikidata...")
uri1 = find_wikidata_entity(term1)
uri2 = find_wikidata_entity(term2)
print(f"Wikidata URI for '{term1}': {uri1}")
print(f"Wikidata URI for '{term2}': {uri2}")

print("\n🔍 Recherche des concepts ORKG...")
orkg1 = find_orkg_entities(term1)
orkg2 = find_orkg_entities(term2)

if not orkg1 or not orkg2:
    print("❌ Aucune entité ORKG trouvée pour au moins un des termes.")
    exit()

start_uri, start_label = orkg1[0]
target_uri, target_label = orkg2[0]
print(f"\n🌐 ORKG Start: {start_label} → {start_uri}")
print(f"🌐 ORKG Target: {target_label} → {target_uri}")

print("\n🔄 Construction du graphe de recherche (max profondeur 3)...")
graph, paths = build_graph_from(start_uri, target_uri, depth=3)

if paths:
    print(f"✅ {len(paths)} chemin(s) trouvé(s) entre les concepts ORKG.")
    for path in paths:
        labels = [graph.nodes[n].get("label", n.split('/')[-1]) for n in path]
        print(" → ".join(labels))
else:
    print("❌ Aucun chemin trouvé jusqu'à la profondeur maximale.")

print("\n🎨 Génération de la visualisation HTML...")
visualize_graph(graph)
print("✅ Graph visualisé dans 'orkg_graph.html'")


🔍 Recherche Wikidata...
Wikidata URI for 'photosynthesis': http://www.wikidata.org/entity/Q11982
Wikidata URI for 'artificial intelligence': http://www.wikidata.org/entity/Q11660

🔍 Recherche des concepts ORKG...


/home/tristan/.venv/lib/python3.11/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html; charset=UTF-8' returning raw response...
  warnings.warn(


TypeError: byte indices must be integers or slices, not str